In [24]:
from datetime import datetime

def archive_paper_under_date(filename, title, authors, institutions, tags, abstract, link, notes, teaser_image_path=None, pipeline_image_path=None):
    """
    Archives a paper record in a Markdown file under the current date's first-level header.
    If the header for today's date does not exist, it's created.
    """
    current_date = datetime.now().strftime("%Y-%m-%d")
    header = f"# {current_date}\n\n"
    paper_record = generate_paper_record(title, authors, institutions, tags, abstract, link, notes, teaser_image_path, pipeline_image_path)
    
    try:
        with open(filename, "r+", encoding="utf-8") as file:
            content = file.readlines()
            file.seek(0)
            if content and content[0].strip() == header.strip():
                # If the first header is today's date, archive under this header
                content.insert(1, paper_record + "\n")
            else:
                # Otherwise, prepend today's header and the record
                content = [header] + [paper_record + "\n"] + content
            file.writelines(content)
    except FileNotFoundError:
        # If the file does not exist, create it with the header and the record
        with open(filename, "w", encoding="utf-8") as file:
            file.write(header + paper_record + "\n")

def generate_paper_record(title, authors, institutions, tags, abstract, link, notes, teaser_image_path, pipeline_image_path):
    """
    Generates the Markdown text for a paper record.
    """
    record = f"## {title}\n\n"
    record += f"- **Authors**: {', '.join(authors)}\n"
    record += f"- **Institutions**: {', '.join(institutions)}\n"
    tags_formatted = ', '.join([f"`{tag}`" for tag in tags])
    record += f"- **Tags**: {tags_formatted}\n\n"
    
    if teaser_image_path:
        record += f"![Teaser Image]({teaser_image_path})\n\n"
    
    if pipeline_image_path:
        record += f"![Pipeline Image]({pipeline_image_path})\n\n"
    
    record += f"### Abstract\n\n{abstract}\n\n"
    record += f"[Paper PDF]({link})\n\n"
    record += f"### Notes\n\n{notes}\n\n"
    
    return record

def add_date_header(filename):
    """
    Adds a date header to the Markdown file if not already present for today, using a second-level header.
    Returns True if a new header was added, otherwise False.
    """
    current_date = datetime.now().strftime("%Y-%m-%d")
    header = f"# {current_date}\n\n"  # Use first-level header for dates
    
    try:
        with open(filename, "r+", encoding="utf-8") as file:
            content = file.read()
            if header.strip() not in content:
                # If today's date header not in file, prepend it
                file.seek(0, 0)  # Move the cursor to the beginning of the file
                file.write(header + content)
                return True
            return False
    except FileNotFoundError:
        # If the file does not exist, create it and add the header
        with open(filename, "w", encoding="utf-8") as file:
            file.write(header)
        return True
def add_date_header2(filename):
    current_date = datetime.now().strftime("%Y-%m-%d")
    header = f"# {current_date}\n\n"
    
    try:
        with open(filename, "r+", encoding="utf-8") as file:
            content = file.read()
            if header.strip() not in content:
                # If today's date header not in file, add it to the beginning
                file.seek(0, 0)  # Move the cursor to the beginning of the file
                file.write(header + content)
    except FileNotFoundError:
        # If the file does not exist, create it and add the header
        with open(filename, "w", encoding="utf-8") as file:
            file.write(header)

def add_paper_with_images(title, authors, institutions, tags, abstract, link, notes, filename, teaser_image_path=None, pipeline_image_path=None):
    """
    Adds a paper record to a Markdown file, including teaser and pipeline images if provided. Automatically
    categorizes the paper under the current date's first-level header.
    """
    is_new_day = add_date_header(filename=filename)  # Check and add the date header if necessary
    
    with open(filename, "a", encoding="utf-8") as file:
        if not is_new_day:
            # If not the first entry of the day, add a separator for readability
            file.write("\n---\n\n")
            
        # Paper title
        file.write(f"## {title}\n\n")  # Using second-level header for paper titles
        
        # Authors and institutions
        file.write(f"- **Authors**: {', '.join(authors)}\n")
        file.write(f"- **Institutions**: {', '.join(institutions)}\n")
        
        # Tags
        formatted_tags = ', '.join([f"`{tag}`" for tag in tags])
        file.write(f"- **Tags**: {formatted_tags}\n\n")
        
        # Teaser image, if provided
        if teaser_image_path:
            file.write(f"![Teaser Image]({teaser_image_path})\n\n")
        
        # Pipeline image, if provided
        if pipeline_image_path:
            file.write(f"![Pipeline Image]({pipeline_image_path})\n\n")
        
        # Abstract
        file.write(f"### Abstract\n\n{abstract}\n\n")
        
        # Link
        file.write(f"[Paper PDF]({link})\n\n")
        
        # Notes
        file.write(f"### Notes\n\n{notes}\n\n")
def add_paper_with_images2(title, authors, institutions, tags, abstract, link, notes, filename, teaser_image_path=None, pipeline_image_path=None):
    """
    Adds a paper record to a Markdown file, including teaser and pipeline images if provided.
    """
    add_date_header(filename=filename)  # Ensure the file has today's date header
    
    with open(filename, "a", encoding="utf-8") as file:
        # Paper title
        file.write(f"## {title}\n\n")
        
        # Authors and institutions
        file.write(f"- **Authors**: {', '.join(authors)}\n")
        file.write(f"- **Institutions**: {', '.join(institutions)}\n")
        
        # Tags
        formatted_tags = ', '.join([f"`{tag}`" for tag in tags])
        file.write(f"- **Tags**: {formatted_tags}\n\n")
        
        # Teaser image, if provided
        if teaser_image_path:
            file.write(f"![Teaser Image]({teaser_image_path})\n\n")
        
        # Pipeline image, if provided
        if pipeline_image_path:
            file.write(f"![Pipeline Image]({pipeline_image_path})\n\n")
        
        # Abstract
        file.write(f"## Abstract\n\n{abstract}\n\n")
        
        # Link
        file.write(f"## Link\n\n[Paper PDF]({link})\n\n")
        
        # Notes
        file.write(f"## Notes\n\n{notes}\n\n")


def add_tag(tag, description, filename="data/tags.txt"):
    """
    向标签文件添加一个新的标签。
    """
    with open(filename, "a", encoding="utf-8") as file:
        file.write(f"{tag}: {description}\n")


def add_institution(name, abbreviation, filename="data/institutions.txt"):
    """
    向机构文件添加一个新的机构。
    """
    with open(filename, "a", encoding="utf-8") as file:
        file.write(f"{abbreviation}: {name}\n")




In [23]:
filename="data/papers.md"
add_date_header(filename)

True

In [25]:
if __name__ == "__main__":
    archive_paper_under_date(
        filename="data/papers.md",
        title="Example Paper Title",
        authors=["Author 1", "Author 2"],
        institutions=["Institution 1", "Institution 2"],
        tags=["nav", "sim2real"],
        abstract="This is the abstract.",
        link="http://example.com",
        notes="These are the notes.",
        teaser_image_path="http://example.com/teaser.jpg",
        pipeline_image_path=None
    )

In [22]:
# 示例使用

# 添加一篇论文记录
add_paper_with_images(
    title="Example Paper Title",
    authors=["Author 1", "Author 2"],
    institutions=["Institution 1", "Institution 2"],
    tags=["nav", "sim2real"],
    abstract="This is the abstract.",
    link="http://example.com",
    notes="These are the notes.",
    filename=filename,
    teaser_image_path="http://example.com/teaser.jpg",  # Or local path "./images/teaser.jpg"
    pipeline_image_path=""  # Leave empty if not available   
)


# 添加标签
add_tag("nav", "导航")
add_tag("sim2real", "从仿真到现实")

# 添加机构
add_institution("北京大学", "PKU")
add_institution("慕尼黑技术大学", "TUM")
